In [ ]:
import tensorflow as tf
from model import build_model
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.optimizers import Adam

def test_backbone(backbone_fn, lr):
    # monta un modelo rápido para 1 época
    model = backbone_fn(num_classes=3)
    model.compile(optimizer=Adam(lr), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    train_ds = tf.keras.preprocessing.image_dataset_from_directory("data/processed/train", image_size=(224,224), batch_size=32)
    val_ds   = tf.keras.preprocessing.image_dataset_from_directory("data/processed/val",   image_size=(224,224), batch_size=32)
    hist = model.fit(train_ds, validation_data=val_ds, epochs=1)
    return hist.history["val_accuracy"][-1]

results = {}
for fn,name in [(build_model,"MobileNetV2"), (lambda nc: build_model(nc, head_units=128), "EffNetB0")]:
    for lr in [1e-3,1e-4]:
        key = f"{name}_lr{lr}"
        results[key] = test_backbone(fn, lr)
import pandas as pd
pd.Series(results).plot.bar()
plt.ylabel("Val Accuracy")
plt.show()